# Ideas
- more generalizable version of _apply so that any op can be applied to obj, not just mapping some func on self.vals
    - inplace param also needs to generalize
- figure out how I want decorator to work (or no decorator?). Don't love current implementation of having to return a lambda
    - maybe chainable should be a method of Chained?
- still have to think about how this could be used: mixin class? Would a metaclass be useful here? Have to think.

In [1]:
from functools import wraps, partial

In [53]:
def chain(func):
    @wraps(func)
    def wrapped(instance, *args, **kwargs):
        return instance._apply(func(*args, **kwargs))
    return wrapped

In [126]:
class Chainable:
   
    def __init__(self, vals):
        self.vals = vals
        self.ops = []
    
    def _apply(self, func, mode='map'):
        op = map if mode == 'map' else filter
        self.ops.append((op, func))
        return self
        
    def exec(self, inplace=False):
        new = self
        for op, func in self.ops:
            new = Chainable(list(op(func, new.vals)))
        self.ops.clear()
        if inplace: 
            self.__dict__ = new.__dict__
        else: 
            return new
        
    def double(self):
        return self._apply(lambda x: x*2)
        
    def add(self, n):
        return self._apply(lambda x: x+n)
        
    def stringify(self):
        return self._apply(str)
        
    def gt(self, n):
        return self._apply(lambda x: x > n, 'filter')
        
    def even_only(self):
        return self._apply(lambda x: x % 2 == 0, 'filter')
    
    @chain
    def subtract(n):
        return lambda x: x-n
        
    def __repr__(self):
        return f'Chainable({repr(self.vals)})'

In [127]:
c = Chainable([1, 3, 2, 44, -5, -7, 16, 6.6, 9, -3, 0.5, 1])
c

Chainable([1, 3, 2, 44, -5, -7, 16, 6.6, 9, -3, 0.5, 1])

In [128]:
c.double().exec()

Chainable([2, 6, 4, 88, -10, -14, 32, 13.2, 18, -6, 1.0, 2])

In [129]:
c.double().even_only().gt(-5).stringify().exec()

Chainable(['2', '6', '4', '88', '32', '18', '2'])

In [130]:
c.subtract(4).double().exec()

Chainable([-6, -2, -4, 80, -18, -22, 24, 5.199999999999999, 10, -14, -7.0, -6])

In [131]:
c

Chainable([1, 3, 2, 44, -5, -7, 16, 6.6, 9, -3, 0.5, 1])

In [123]:
c.add(4).subtract(2).even_only().gt(-1).exec(inplace=True)

In [124]:
c

Chainable([4, 46, 18])

## Working on more generalized version

Current implementation: user defines a staticmethod for each chainable op, then an instance method that calls that staticmethod. This works but is annoying to use.

In [2]:
from copy import copy, deepcopy
from functools import update_wrapper, wraps
import inspect

In [816]:
class Chainable():
   
    def __init__(self, vals, num):
        self.vals = vals
        self.been_called = False
        self.num = num
        self.ops = []
    
    def _apply(self, func):
        self.ops.append(func)
        return self
        
    def exec(self, inplace=False):
        new = deepcopy(self)
        for func in self.ops:
            new = func(copy(new))

        # Clear ops list now that chain is complete.
        new.ops.clear()
        if inplace: 
            self.__dict__ = new.__dict__
        else: 
            self.ops.clear()
            return new
        
    @staticmethod
    def _double(instance):
        instance.num *= 2
        return instance
    
    def double(self):
        return self._apply(self._double)
        
    @staticmethod
    def _add(instance, n):
        instance.num += n
        return instance
    
    def add(self, n):
        return self._apply(partial(self._add, n=n))

    @staticmethod    
    def _append(instance, n):
        instance.vals.append(n)
        return instance

    def append(self, n):
        return self._apply(partial(self._append, n=n))
        
    @staticmethod
    def _call(instance):
        instance.been_called = True
        return instance
    
    def call(self):
        return self._apply(self._call)
    
    def __repr__(self):
        return f'Chainable({repr(self.vals)}, {self.been_called}, {self.num}, {self.ops})'

In [817]:
c1 = Chainable([1, 3], 5)
c1

Chainable([1, 3], False, 5, [])

In [818]:
c2 = copy(c1)
c2

Chainable([1, 3], False, 5, [])

In [819]:
id(c1.vals), id(c2.vals)

(4395407944, 4395407944)

In [820]:
c2.vals.append(3)
c1, c2

(Chainable([1, 3, 3], False, 5, []), Chainable([1, 3, 3], False, 5, []))

In [821]:
c1.double().exec(True)

In [822]:
c1, c2

(Chainable([1, 3, 3], False, 10, []),
 Chainable([1, 3, 3], False, 5, [<function Chainable._double at 0x106a9cc80>]))

In [823]:
c2.append(-1).exec(True)

In [824]:
c1, c2

(Chainable([1, 3, 3], False, 10, []), Chainable([1, 3, 3, -1], False, 10, []))

### Testing basic func

In [828]:
vals = [5, 3, 1, 2, 4, 6]
n = 44
length = len(vals)

In [829]:
c = Chainable(vals, n)
c

Chainable([5, 3, 1, 2, 4, 6], False, 44, [])

In [830]:
c.add(3).double().exec()

Chainable([5, 3, 1, 2, 4, 6], False, 94, [])

In [831]:
print(c)

assert c.vals == vals and not c.been_called and c.num == n and not c.ops

Chainable([5, 3, 1, 2, 4, 6], False, 44, [])


In [832]:
c.add(3).double().exec(inplace=True)

In [833]:
print(c)
assert c.vals == vals and not c.been_called and c.num == (n+3)*2 and not c.ops

Chainable([5, 3, 1, 2, 4, 6], False, 94, [])


### Testing effect on mutable attrs

In [834]:
c.append(99).call().exec(False)

Chainable([5, 3, 1, 2, 4, 6, 99], True, 94, [])

In [835]:
# Bool attr is unchanged (as desired), but list is changed.
print(c)
assert c.vals == vals and not c.been_called and c.num == (n+3)*2 and not c.ops

Chainable([5, 3, 1, 2, 4, 6], False, 94, [])


In [836]:
c.append(99).call().exec(True)

In [837]:
# Bool attr and list attr both changed.
print(c)
assert c.vals == vals+[99] and c.been_called and c.num == (n+3)*2 and not c.ops

Chainable([5, 3, 1, 2, 4, 6, 99], True, 94, [])


In [838]:
assert len(vals) == length

## Testing meta magic

Idea: can we make it simpler or more intuitive to define chainable methods?

Status: Tentatively working, needs testing though.

In [3]:
import types

from htools import hdir, debug

In [236]:
# class ChainMethod:
    
#     def __init__(self, func):
#         wraps(func)(self)
        
#     def __call__(self, *args, **kwargs):
#         print('call', args, kwargs)
#         return self.__wrapped__(*args, **kwargs)
    
#     def __get__(self, instance, cls):
#         if instance is None:
#             return self
#         return types.MethodType(self, instance)

    
def chain(func):
    func._is_chainable = True
    
    @wraps(func)
    def wrapped(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapped


# def ChainMethod(func):
#     # This version does attach _is_trainable attr but v.__name__
#     # is now wrapper in metaclass
    
#     @wraps(func)
#     def wrapper(*args, **kwargs):
#         print('CALL DECORATOR: args', args, '\nkwargs', kwargs)
#         return func(*args, **kwargs)
#     static = staticmethod(wrapper)
#     static._is_chainable = True
#     return static

In [237]:
class ChainMeta(type):
    
    def __new__(cls, name, bases, methods):
        new_methods = {}
        
        for k, v in methods.items():
            try: 
                func = v.__get__(1)
                assert func._is_chainable
            except: 
                continue    
            public_name = k.lstrip('_')
            
            # Capture args and kwargs passed to staticmethod (except for instance).
            sig = inspect.signature(func)
            sig = sig.replace(parameters=list(sig.parameters.values())[1:])
            
            # Must use default args, otherwise func will always point to last method.
            def make_public_method(func=func, private_name=k, 
                                   public_name=public_name, sig=sig):
                def public(inst, *args, **kwargs):
                    bound = sig.bind(*args, **kwargs).arguments
                    new_method = partial(getattr(inst, private_name), **bound)
                    inst.ops.append(new_method)
                    return inst
                public.__name__ = public_name
                return public
            
            new_methods[public_name] = make_public_method()
                
        return type.__new__(cls, name, bases, {**methods, **new_methods})

In [238]:
class Chainable(metaclass=ChainMeta):
   
    def __init__(self, vals, num):
        self.vals = vals
        self.been_called = False
        self.num = num
        self.ops = []
    
    def _apply(self, func):
        self.ops.append(func)
        return self
        
    def exec(self, inplace=False):
        new = deepcopy(self)
        for func in self.ops:
            new = func(copy(new))
        # Clear ops list now that chain is complete.
        new.ops.clear()
        if inplace: 
            self.__dict__ = new.__dict__
        else: 
            self.ops.clear()
            return new
    
    @staticmethod
    @chain
    def _call(instance):
        instance.been_called = True
        return instance
    
    @staticmethod
    @chain
    def _incr(instance):
        instance.num += 1
        return instance
    
    @staticmethod
    @chain
    def _append(instance, n):
        instance.vals.append(n)
        return instance
    
    def __repr__(self):
        return f'Chainable({repr(self.vals)}, {self.been_called}, {self.num}, {self.ops})'

In [239]:
c1 = Chainable([1, 3], 5)
c1

Chainable([1, 3], False, 5, [])

In [240]:
assert not c1.been_called
_ = c1.call().exec(True)
assert _ is None
assert c1.been_called

In [241]:
assert c1.num == 5
c1.incr().exec()
assert c1.num == 5

In [242]:
_ = c1.incr().exec(True)
assert _ is None
assert c1.num == 6

In [243]:
c1 = Chainable([2, 4, 6], 111)
c1

Chainable([2, 4, 6], False, 111, [])

In [244]:
c2 = c1.incr().call().exec()
assert c2.been_called
assert c2.num == 112

assert not c1.been_called
assert c1.num == 111

In [245]:
_ = c1.incr().call().append(n=99).exec(True)
assert _ is None
assert c1.vals == [2, 4, 6, 99]
assert c1.been_called
assert c1.num == 112

### Issue: only kwargs work atm, not args. Must map args to correct params in partial in metaclass

In [246]:
c2 = c1.append(4).exec()
assert c2.vals[-1] == 4
assert c1.vals[-1] == 99

In [247]:
c1

Chainable([2, 4, 6, 99], True, 112, [])

In [248]:
c1.append(333).incr().call().exec()

Chainable([2, 4, 6, 99, 333], True, 113, [])

In [249]:
c1

Chainable([2, 4, 6, 99], True, 112, [])

In [250]:
c1.append(333).incr().call().exec(True)

In [251]:
c1

Chainable([2, 4, 6, 99, 333], True, 113, [])

## Testing finished lazy chainable

Note: Library version has a few changes to formatting (line lengths), docstrings, and removed \_apply() method in an effort to enforce 1 correct way of making chainable methods. chain decorator is also renamed to lazychain.

### IDEA: check if staticmethod is actually necessary now. May have been left over from a previous approach.

In [320]:
def chain(func):
    """Decorator to register a method as chainable within a
    LazyChainable class.
    """
    func._is_chainable = True
    
    @wraps(func)
    def wrapped(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapped

In [319]:
class LazyChainMeta(type):
    """Metaclass to create LazyChainable objects."""
    
    def __new__(cls, name, bases, methods):
        new_methods = {}
        
        # Find chainable staticmethods and create public versions.
        for k, v in methods.items():
            try: 
                func = v.__get__(1)
                assert func._is_chainable
            except: 
                continue    
            public_name = k.lstrip('_')
            
            # Capture args and kwargs passed to staticmethod (except for instance).
            sig = inspect.signature(func)
            sig = sig.replace(parameters=list(sig.parameters.values())[1:])
            
            # Must use default args, otherwise func will always point to last method.
            def make_public_method(func=func, private_name=k, 
                                   public_name=public_name, sig=sig):
                def public(inst, *args, **kwargs):
                    bound = sig.bind(*args, **kwargs).arguments
                    new_method = partial(getattr(inst, private_name), **bound)
                    inst.ops.append(new_method)
                    return inst
                public.__name__ = public_name
                return public
            
            new_methods[public_name] = make_public_method()
                
        return type.__new__(cls, name, bases, {**methods, **new_methods})

In [318]:
class LazyChainable(metaclass=LazyChainMeta):
    """Base class that allows children to lazily chain methods, 
    similar to a Spark RDD.
    
    Chainable methods must be decorated with @staticmethod
    and @chain and be named with a leading underscore. A public
    method without the leading underscore will be created, so don't
    overwrite this with another method. Chainable methods
    accept an instance of the same class as the first argument, 
    process the instance in some way, then return it. A chain of 
    commands will be stored until the exec() method is called.
    It can operate either in place or not.
    
    Examples
    --------
    class Sequence(LazyChainable):
    
        def __init__(self, numbers, counter, new=True):
            super().__init__()
            self.numbers = numbers
            self.counter = counter
            self.new = new

        @staticmethod
        @chain
        def _sub(instance, n):
            instance.counter -= n
            return instance

        @staticmethod
        @chain
        def _gt(instance, n=0):
            instance.numbers = list(filter(lambda x: x > n, instance.numbers))
            return instance

        @staticmethod
        @chain
        def _call(instance):
            instance.new = False
            return instance

        def __repr__(self):
            pre, suf = super().__repr__().split('(')
            argstrs = (f'{k}={repr(v)}' for k, v in vars(self).items())
            return f'{pre}({", ".join(argstrs)}, {suf}'
    
    
    >>> seq = Sequence([3, -1, 5], 0)
    >>> output = seq.sub(n=3).gt(0).call().exec()
    >>> output
    
    Sequence(ops=[], numbers=[3, 5], counter=-3, new=False)
    
    >>> seq   # Unchanged because exec was not in place.
    
    Sequence(ops=[], numbers=[3, -1, 5], counter=0, new=True)
    
    
    >>> output = seq.sub(n=3).gt(-1).call().exec(inplace=True)
    >>> output   # None because exec was in place.
    >>> seq   # Changed
    
    Sequence(ops=[], numbers=[3, -1, 5], counter=-3, new=False)
    """
   
    def __init__(self):
        self.ops = []
    
    def _apply(self, func):
        self.ops.append(func)
        return self
        
    def exec(self, inplace=False):
        new = deepcopy(self)
        for func in self.ops:
            new = func(copy(new))
        # Clear ops list now that chain is complete.
        new.ops.clear()
        if inplace: 
            self.__dict__ = new.__dict__
        else: 
            self.ops.clear()
            return new
    
    def __repr__(self):
        argstrs = (f'{k}={repr(v)}' for k, v in vars(self).items())
        return f'{type(self).__name__}({", ".join(argstrs)})'

In [314]:
class Sequence(LazyChainable):
    
    def __init__(self, numbers, counter, new=True):
        super().__init__()
        self.numbers = numbers
        self.counter = counter
        self.new = new
        
    @staticmethod
    @chain
    def _sub(instance, n):
        instance.counter -= n
        return instance
    
    @staticmethod
    @chain
    def _product(instance):
        prod = 1
        for num in instance.numbers:
            prod *= num
        instance.prod = prod
        return instance
    
    @staticmethod
    @chain
    def _gt(instance, n=0):
        instance.numbers = list(filter(lambda x: x > n, instance.numbers))
        return instance
    
    @staticmethod
    @chain
    def _call(instance):
        instance.new = False
        return instance
    
    def __repr__(self):
        return super().__repr__()

In [297]:
seq = Sequence([1, 11, 99, 4, -3, -0.5, 1.5, -22.2], 4, new=True)
seq

Sequence(ops=[], numbers=[1, 11, 99, 4, -3, -0.5, 1.5, -22.2], counter=4, new=True, ops=[])

In [298]:
seq.product().exec()

Sequence(ops=[], numbers=[1, 11, 99, 4, -3, -0.5, 1.5, -22.2], counter=4, new=True, prod=-217582.19999999998, ops=[])

In [299]:
seq.gt(4).sub(3).product().exec(True)
seq

Sequence(ops=[], numbers=[11, 99], counter=1, new=True, prod=1089, ops=[])

In [300]:
seq.ops

[]

In [301]:
seq.sub(n=4).call().product().exec()

Sequence(ops=[], numbers=[11, 99], counter=-3, new=False, prod=1089, ops=[])

In [302]:
isinstance(seq, LazyChainable)

True

In [303]:
type(LazyChainable)

__main__.LazyChainMeta

## Try another approach using \_\_getattr\_\_ instead of metaclass

In [4]:
def ChainMethod(func):
    func._is_chainable = True
    
    @wraps(func)
    def wrapped(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapped

In [75]:
class Chainable:
   
    def __init__(self, vals, num):
        self.vals = vals
        self.been_called = False
        self.num = num
        self.ops = []
        
    def __getattr__(self, name, *args, **kwargs):
        print('args, kwargs', args, kwargs)
        private_name = '_' + name
        method = super().__getattribute__(private_name)
        if method._is_chainable:
            self._apply(method)
        return method
    
    def _apply(self, func):
        self.ops.append(func)
        return self
        
    def exec(self, inplace=False):
        new = deepcopy(self)
        for func in self.ops:
            new = func(copy(new))
        # Clear ops list now that chain is complete.
        new.ops.clear()
        if inplace: 
            self.__dict__ = new.__dict__
        else: 
            self.ops.clear()
            return new
    
    @staticmethod
    @ChainMethod
    def _call(instance):
        instance.been_called = True
        return instance
    
    @staticmethod
    @ChainMethod
    def _incr(instance):
        instance.num += 1
        return instance
    
    @staticmethod
    @ChainMethod
    def _append(instance, n):
        instance.vals.append(n)
        return instance
    
    def __repr__(self):
        return f'Chainable({repr(self.vals)}, {self.been_called}, {self.num}, {self.ops})'

In [76]:
c = Chainable([2, 4, 0], 100)
c

args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}
args, kwargs () {}


Chainable([2, 4, 0], False, 100, [])

In [77]:
c.incr()

args, kwargs () {}


TypeError: _incr() missing 1 required positional argument: 'instance'

In [79]:
c.append(n=3)

args, kwargs () {}


TypeError: _append() missing 1 required positional argument: 'instance'